In [49]:
import pyaudio #If you have issues with this, see the powerpoint for help 
import pyttsx3
import speech_recognition as sr
import os

import re, requests, subprocess, urllib.parse, urllib.request
from bs4 import BeautifulSoup

import pafy
import vlc
import time

In [34]:
# Initializing microphone and speech recognition system
mic = sr.Microphone()


In [35]:
# Using the simplest voice engine we can to do text to speech
engine = pyttsx3.init()
voices = engine.getProperty('voices') # if you get a weird accent, try printing this variable and choose another voice
engine.setProperty('voice', voices[0].id) # a different voice id might be necessary on your system

print(voices)

[<pyttsx3.voice.Voice object at 0x0000017180137E50>, <pyttsx3.voice.Voice object at 0x0000017180762F10>]


In [36]:
#Check the files can be opened: 

ing_list = """150 grams of butter 
100 grams of brown sugar 
100 grams of cane sugar
200 grams of flour
100 grams of dark chocolate
100 grams of milk chocolate
100 grams of white chocolate. Yes. You want that much chocolate. 
1 egg
10 grams of vanilla  
1 teaspoon of baking powder"""

ing_list = ing_list.split("\n")
inst_list = """Preheat the oven to 180 degrees 
Combine the egg with the brown sugar and the cane sugar in a mixing bowl 
Add the the flour, chocolate, vanilla, baking powder, and butter to the mixture 
Stir to combine the ingredients 
Let the dough rest in the fridge till tomorrow... nah, no one can wait that long for cookies 
Devide the dough into golfball sized spheres and place them on baking paper in a cooking tray 
Cook them in the oven for 10-14 minutes 
Eat cookies, mmmmm, very tasty right? 
"""
inst_list = inst_list.split("\n")

cake_list = '''First you take some butter
Then you take some flour
You add some eggs aswell
stir it all together
bake
congratulations
'''
cake_list = cake_list.split("\n")

print(inst_list)
print(ing_list)


['Preheat the oven to 180 degrees ', 'Combine the egg with the brown sugar and the cane sugar in a mixing bowl ', 'Add the the flour, chocolate, vanilla, baking powder, and butter to the mixture ', 'Stir to combine the ingredients ', 'Let the dough rest in the fridge till tomorrow... nah, no one can wait that long for cookies ', 'Devide the dough into golfball sized spheres and place them on baking paper in a cooking tray ', 'Cook them in the oven for 10-14 minutes ', 'Eat cookies, mmmmm, very tasty right? ', '']
['150 grams of butter ', '100 grams of brown sugar ', '100 grams of cane sugar', '200 grams of flour', '100 grams of dark chocolate', '100 grams of milk chocolate', '100 grams of white chocolate. Yes. You want that much chocolate. ', '1 egg', '10 grams of vanilla  ', '1 teaspoon of baking powder']


In [54]:

# Second: Initialize dynamic variables
active_list = ing_list # we start by iterating through the ingredient list
i = 0 # the counter we use to read through our list
textInput = "nothing" # we initialise the input for the loop to work

engine.say("Hi. My name is YouTuPy but you can call me Jam")
print("Hi. My name is YouTuPy but you can call me Jam")
engine.say("If you want to stop our good thing at any point, just say stop")
print("If you want to stop our good thing at any point, just say stop")

# All interactions happen here
while textInput != "stop":

    # We load the speech to text model
    r = sr.Recognizer()
    # check for background noise and adjust sensitivity accordingly
    with mic as source:
        audio = r.adjust_for_ambient_noise(source)

    # Listen for user input
    print("I am listening now. What should I play?")
    engine.say("I am listening now. What should I play?")
    engine.runAndWait()
    with mic as source:
        audio = r.listen(source) # save the mic input into the "audio" variable
    print("processing...")

    try:
        # speech to text: transform the audio recorded from the mic into text
        textInput = r.recognize_google(audio)
        
        if textInput == "stop":
            response = "Well, see you later skater" 
        
        else: 
            print("You said: " + textInput + " Is that what you meant?")
            engine.say("You said: " + textInput + " Is that what you meant?")
            
            # We load the speech to text model
            r = sr.Recognizer()
            # check for background noise and adjust sensitivity accordingly
            with mic as source:
                audio = r.adjust_for_ambient_noise(source)
            
            print("I am listening now... please confirm (yes/no)")
            engine.say("I am listening now... please confirm")
            engine.runAndWait()    
            with mic as source:
                audio = r.listen(source) # save the mic input into the "audio" variable
            print("processing...")
            
            try: 
                textInput2 = r.recognize_google(audio)

                if textInput2 != "yes":
                    break 
                    
                else: 
                    music_name = textInput
                    query_string = urllib.parse.urlencode({"search_query": music_name})
                    formatUrl = urllib.request.urlopen("https://www.youtube.com/results?" + query_string)
                    search_results = re.findall(r"watch\?v=(\S{11})", formatUrl.read().decode())
                    clip = requests.get("https://www.youtube.com/watch?v=" + "{}".format(search_results[0]))
                    clip2 = "https://www.youtube.com/watch?v=" + "{}".format(search_results[0])
                    inspect = BeautifulSoup(clip.content, "html.parser")
                    yt_title = inspect.find_all("meta", property="og:title")
                    
                    for concatMusic1 in yt_title:
                        pass

                    print("Does this sound right?: " + concatMusic1['content'])
                    engine.say("Does this sound right?:" + concatMusic1['content'])
                    
                    
                    # We load the speech to text model
                    r = sr.Recognizer()
                    # check for background noise and adjust sensitivity accordingly
                    with mic as source:
                        audio = r.adjust_for_ambient_noise(source)

                    engine.runAndWait()    
                    with mic as source:
                        audio = r.listen(source) # save the mic input into the "audio" variable
                    print("processing...")

                    try: 
                        textInput3 = r.recognize_google(audio)

                        if textInput3 == "yes": 
                    
                            url = clip2
                            video = pafy.new(url)
                            best = video.getbest()
                            playurl = best.url

                            Instance = vlc.Instance()
                            player = Instance.media_player_new()
                            Media = Instance.media_new(playurl)
                            Media.get_mrl()
                            player.set_media(Media)
                            player.play()

                            timeout = time.time() + 10   # 3 seconds from now
                            while True:
                                if time.time() > timeout:
                                    player.stop()
                                    
                    # API was unreachable or unresponsive:
                    except sr.RequestError:
                        response = "API unavailable"
                    # if the speech was unintelligible:
                    except sr.UnknownValueError:
                        response = "Sorry! Something went wrong with your mouth. " \
                                   "Please, try to be intelligible. "
                    # if you arrive at the end of the list:
                    except IndexError:
                        response = "There is nothing left in this list. " \
                                   "You can ask me for the list of Ingredients by saying Ingredients. " \
                                    "You can ask me for the list of instructions by saying Instructions. "
                
                
            # API was unreachable or unresponsive:
            except sr.RequestError:
                response = "API unavailable"
            # if the speech was unintelligible:
            except sr.UnknownValueError:
                response = "Sorry! Something went wrong with your mouth. " \
                           "Please, try to be intelligible. "
            # if you arrive at the end of the list:
            except IndexError:
                response = "There is nothing left in this list. " \
                           "You can ask me for the list of Ingredients by saying Ingredients. " \
                            "You can ask me for the list of instructions by saying Instructions. "
            
    # API was unreachable or unresponsive:
    except sr.RequestError:
        response = "API unavailable"
    # if the speech was unintelligible:
    except sr.UnknownValueError:
        response = "Sorry! Something went wrong with your mouth. " \
                   "Please, try to be intelligible. "
    # if you arrive at the end of the list:
    except IndexError:
        response = "There is nothing left in this list. " \
                   "You can ask me for the list of Ingredients by saying Ingredients. " \
                    "You can ask me for the list of instructions by saying Instructions. "


    print(response)
    engine.say(response)
    engine.runAndWait()

Hi. My name is YouTuPy but you can call me Jam
If you want to stop our good thing at any point, just say stop
I am listening now. What should I play?
processing...
You said: Kashmir Is that what you meant?
I am listening now... please confirm (yes/no)
processing...
Does this sound right?: Kashmir - Mom In Love, Daddy In Space
processing...


KeyboardInterrupt: 